In [8]:
from data_utils import LanesDataset, BDD100k
from torch.utils.data import DataLoader

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [139]:
class convBlock(nn.Module):
    """Some Information about convBlock"""
    def __init__(self, in_channels, out_channels, ks=3, stride=1, padding=1):
        super(convBlock, self).__init__()
        
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, ks, stride, padding)
        

    def forward(self, x):
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv(x)
        return x

In [165]:
class denseBlock2(nn.Module):
    """Some Information about denseBlock2"""
    def __init__(self, in_channels, out_channels, growth_rate, num_convs):
        super(denseBlock2, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.growth_rate = growth_rate
        self.net = nn.Sequential()
        for i in range(num_convs):
            self.net.add_module(f'conv{i}', convBlock(self.in_channels, self.growth_rate))
            self.in_channels = self.in_channels + self.growth_rate
        self.out_conv = convBlock(self.in_channels, self.out_channels, ks=3)

    def forward(self, x):
        for blk in self.net.children():
            y = blk(x)
            x = torch.cat([x, y], 1)
        return self.out_conv(x)


In [176]:
class denseBlockUp(nn.Module):
    """Some Information about denseBlockUp"""
    def __init__(self, in_channels, out_channels, growth_rate, num_convs):
        super(denseBlockUp, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.growth_rate = growth_rate
        self.net = nn.Sequential()
        for i in range(num_convs):
            self.net.add_module(f'conv{i}', convBlock(self.in_channels, self.growth_rate))
            self.in_channels = self.in_channels + self.growth_rate
        self.out_conv = convBlock(self.in_channels, self.out_channels, ks=3)

    def forward(self, x):
        for blk in self.net.children():
            y = blk(x)
            x = torch.cat([x, y], 1)
            print(x.shape)
        return self.out_conv(x)

In [6]:
train_img_dir = 'datasets/bdd100k/images/100k/train/'
train_label_dir = 'datasets/bdd100k/drivable_maps/labels/train/'
val_img_dir = 'datasets/bdd100k/images/100k/val/'
val_label_dir = 'datasets/bdd100k/drivable_maps/labels/val/'
model_dir = 'saved_models/'
exp_no = 2
logdir = 'runs/BDD100k_Experiment'+str(exp_no)
resize = (128,128)
SAMPLE_SIZE = 2000

In [9]:
# loading train data
train_dataset = BDD100k(train_img_dir,train_label_dir,resize=resize, transform=True, grayscale=False)
train_dataloader = DataLoader(train_dataset,batch_size=1, shuffle=False, pin_memory=True, num_workers=2)


In [16]:
img, mask = iter(train_dataloader).next()
print(img.shape)
print(mask.shape)

torch.Size([1, 2, 128, 128])

In [167]:
conv = nn.Conv2d(3,32,3,stride=1,padding=1)
# print(conv)
x = conv(img)
dense = denseBlock2(32, 64, 32, 3)
y = dense(x)
print(dense)
print(y.shape)


denseBlock2(
  (net): Sequential(
    (conv0): convBlock(
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv1): convBlock(
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): convBlock(
      (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (out_conv): convBlock(
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)
torch.Size([1, 64, 128, 128])


In [177]:
conv = nn.Conv2d(3,64,3,stride=1,padding=1)
# print(conv)
x = conv(img)
dense = denseBlockUp(64, 32, 16, 2)
y = dense(x)
print(dense)
print(y.shape)

torch.Size([1, 80, 128, 128])
torch.Size([1, 96, 128, 128])
denseBlockUp(
  (net): Sequential(
    (conv0): convBlock(
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv1): convBlock(
      (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv): Conv2d(80, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (out_conv): convBlock(
    (bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (conv): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)
torch.Size([1, 32, 128, 128])


In [ ]:
from models.covidNet import covidNet